# Retrieve NREL DCFC Locations

In [1]:
#Import requests and pandas packages to our Python environment
import requests
import pandas as pd

In [2]:
#Read key file in
key = open('NRELkey.txt','r').readline()

In [3]:
#Construct the service request
baseURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?'
params = {
    'api_key':key,                #Our key, to authenticate the service
    'status':'E',                 #Request only currently open sites 
    'fuel_type':'ELEC',           #Request only Electric sites
    'ev_charging_level':'dc_fast',#Request only DCFC sites
    'state':'NC' ,                #Request only NC sites
    'access':'public',            #Request only Public sites
    'groups_with_access_code':'Public', #Request only sites open to Public
    'limit':'all'                 #Don't cap results to first 200 (default)
}

In [4]:
#Execute the request, converting the response to a dataframe
df_ALL = pd.DataFrame.from_dict(requests.get(baseURL,params).json()['fuel_stations'])
print(f'{len(df_ALL)} records retrieved')

86 records retrieved


In [5]:
#Drop excess columns
dropCols = [col for col in df_ALL.columns if col[:3] in ('bd_','cng','hy_','ng_','e85','lng','lpg')]
dropCols += [col for col in df_ALL.columns if col[-2:] == 'fr']
df_ALL.drop(columns = dropCols,inplace=True)

In [6]:
#Remove sites in the Tesla network
df_DCFC = df_ALL.loc[df_ALL['ev_network'] != 'Tesla'].reset_index()

In [7]:
df_DCFC.to_csv('../Data/NREL/DCFC.csv',index=False)